In [1]:
import requests
import json
import pandas as pd
import time
import random
import xmltodict


In [2]:
def QueryEdit(url,keyword,Affiliation=None,daterange=None,start = 0,sort = 'citedby-count'):
    url +='?query={}'.format(keyword)
    if Affiliation!=None:
        url+="+AFFIL({})".format(Affiliation)
    url += "&count={}".format(count)
    url += "&sort={}".format(sort)
    url += "&start={}".format(start)
    return url

In [6]:
conf = open('config.json','r') 
config = json.load(conf)
api_key = config['API_key']
conf.close()
keyw = open('keywords.txt','r')
keywords = keyw.readlines()
keyw.close()
for index in range(len(keywords)):
    keywords[index] = keywords[index].replace('\n','')
print(keywords)
Affiliation = 'University of Auckland'
start = 0
count = 25
step = 25
url = "https://api.elsevier.com/content/search/scopus"
headers=  {'X-ELS-APIKey':api_key}
print(url)


['clean technology', 'clean-energy', 'carbon emissions', 'energy efficient', 'green energy', 'renewable energy', 'smart home', 'geothermal', 'wastewater treatment', 'carbon emissions', 'zero waste', 'energy storage', 'heat recovery', 'green building', 'additive manufacturing', 'waste reduction', 'maritime electrification']
https://api.elsevier.com/content/search/scopus


In [3]:
def TidyJsontoDf(web_json,paper_info_short,column):
    try :
        for paper in web_json['search-results']['entry']:
            try:
                title = paper['dc:title']
                autor = paper['dc:creator']
                doi = paper['prism:doi']
                citedby_count = paper['citedby-count']
                affilname = paper['affiliation'][0]['affilname']
                affiliation_city = paper['affiliation'][0]['affiliation-city']
                affiliation_country = paper['affiliation'][0]['affiliation-country']
            except:
                continue
            paper_data = [[title,autor,doi,citedby_count,affilname,affiliation_city,affiliation_country]]
            paper_info = pd.DataFrame(data=paper_data, columns=column)
            paper_info_short = pd.concat([paper_info_short,paper_info],ignore_index=True)
    except:
        pass
    return paper_info_short

In [7]:
def GetAndTidy(keyword,Affiliation = None,url=url, headers = headers):
    url = QueryEdit(url,keyword,Affiliation = None)
    print(url)
    web = requests.get(url,headers=headers)
    result = web.text
    web_json = json.loads(result)
    print(web_json['search-results']['opensearch:totalResults'])
    epos = int(web_json['search-results']['opensearch:totalResults']) // step +1
    column = ['title','autor','doi','citedby_count','affilname','affiliation-city','affiliation-country']
    paper_info_short = pd.DataFrame(columns=column)
    paper_info_short = TidyJsontoDf(web_json,paper_info_short,column)
    for i in range(1,epos+1):
        
        start = i*25
        if start > 10000: break
        url = "https://api.elsevier.com/content/search/scopus"
        url = QueryEdit(url,keyword,start = start)
        print(url)
        web = requests.get(url,headers=headers)
        if(web.ok):
            result = web.text
            web_json = json.loads(result)
            paper_info_short = TidyJsontoDf(web_json,paper_info_short,column)
        time.sleep(random.random())
    return paper_info_short



In [14]:
keyword = 'geothermal'
url = "https://api.elsevier.com/content/search/scopus"
paper_info = GetAndTidy(keyword,url = url)


https://api.elsevier.com/content/search/scopus?query=geothermal&count=25&sort=citedby-count&start=0
215285
https://api.elsevier.com/content/search/scopus?query=geothermal&count=25&sort=citedby-count&start=25
https://api.elsevier.com/content/search/scopus?query=geothermal&count=25&sort=citedby-count&start=50
https://api.elsevier.com/content/search/scopus?query=geothermal&count=25&sort=citedby-count&start=75
https://api.elsevier.com/content/search/scopus?query=geothermal&count=25&sort=citedby-count&start=100
https://api.elsevier.com/content/search/scopus?query=geothermal&count=25&sort=citedby-count&start=125
https://api.elsevier.com/content/search/scopus?query=geothermal&count=25&sort=citedby-count&start=150
https://api.elsevier.com/content/search/scopus?query=geothermal&count=25&sort=citedby-count&start=175
https://api.elsevier.com/content/search/scopus?query=geothermal&count=25&sort=citedby-count&start=200
https://api.elsevier.com/content/search/scopus?query=geothermal&count=25&sort=ci

In [16]:
paper_info.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4738 entries, 0 to 4737
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   title                4738 non-null   object
 1   autor                4738 non-null   object
 2   doi                  4738 non-null   object
 3   citedby_count        4738 non-null   object
 4   affilname            4738 non-null   object
 5   affiliation-city     4712 non-null   object
 6   affiliation-country  4738 non-null   object
dtypes: object(7)
memory usage: 259.2+ KB


In [13]:
filepath = 'data/'
paper_info.to_csv(filepath+keyword+'.csv',index=False)

In [ ]:
filepath = 'data/'
for keyword in keywords:
    url = "https://api.elsevier.com/content/search/scopus"
    paper_info = GetAndTidy(keyword,url = url)
    paper_info.to_csv(filepath+keyword+'.csv',index=False)

In [ ]:
web_json = json.loads(result)
print(web_json['search-results']['opensearch:totalResults'])
epos = int(web_json['search-results']['opensearch:totalResults']) // 25 +1


In [ ]:
for i in range(1,epos+1):
    start = i*25
    url = "https://api.elsevier.com/content/search/scopus"
    url = QueryEdit(url,keyword,start = start)
    print(url)
    web = requests.get(url,headers=headers)
    print(web.ok)
    result = web.text
    colnum = ['title','autor','doi','citedby_count','affilname','affiliation-city','affiliation-country']
    paper_info_short = pd.DataFrame(columns=colnum)
    



In [ ]:
f = open('scopus_raw_result.json','w+')
json_object = json.dumps(web_json, indent=4)
f.write(json_object)
f.close()


In [ ]:
colnum = ['title','autor','doi','citedby_count','affilname','affiliation-city','affiliation-country']
paper_info_short = pd.DataFrame(columns=colnum)

In [ ]:
paper_info_short = paper_info_short.reset_index(drop=True)
   

In [ ]:
101 // 25

In [ ]:
web_json['search-results']['entry'][0]['affiliation']

In [ ]:
aburl = 'https://api.elsevier.com/content/article/doi/10.1016/j.envpol.2022.120034'
abstract = requests.get(aburl,headers = headers)

In [ ]:
print(abstract.text)
temp = open("temp.txt",'w+')
temp.write(abstract.text)
temp.close()

In [ ]:
my_dict = xmltodict.parse(abstract.text)
my_dict['full-text-retrieval-response']['coredata']['dc:description']